# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [123]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.metrics import f1_score
from razdel import tokenize

from IPython.display import Image
from IPython.core.display import HTML 

In [43]:
data = pd.read_csv("O:\ВШЭ Магистратура\НИС Актуальные проблемы компьютерной лингвистики\Домашние работы\labeled.csv")

train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

y = train.toxic.values
y_test = test.toxic.values

<>:1: SyntaxWarning: "\l" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\l"? A raw string is also an option.
<>:1: SyntaxWarning: "\l" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\l"? A raw string is also an option.
C:\Users\MSI\AppData\Local\Temp\ipykernel_15148\120440491.py:1: SyntaxWarning: "\l" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\l"? A raw string is also an option.
  data = pd.read_csv("O:\ВШЭ Магистратура\НИС Актуальные проблемы компьютерной лингвистики\Домашние работы\labeled.csv")


**Дефолтный токенизатор**

In [44]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train.comment)
X.shape

(12970, 63980)

In [45]:
X_test = vectorizer.transform(test.comment)
X_test.shape

(1442, 63980)

In [46]:
clf = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf.fit(X, y)
default_preds = clf.predict(X_test)
default_f1 = f1_score(y_test, default_preds, average='weighted')

print(classification_report(y_test, default_preds))

              precision    recall  f1-score   support

         0.0       0.80      0.61      0.69       949
         1.0       0.49      0.71      0.58       493

    accuracy                           0.64      1442
   macro avg       0.64      0.66      0.64      1442
weighted avg       0.69      0.64      0.65      1442



**razdel**

In [47]:
def razdel_tokenizer(data):
    return [token.text for token in tokenize(data)]
vectorizer = CountVectorizer(tokenizer=razdel_tokenizer)
X = vectorizer.fit_transform(train.comment)
X.shape

c:\Users\MSI\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(12970, 64923)

In [48]:
X_test = vectorizer.transform(test.comment)
X_test.shape

(1442, 64923)

In [49]:
clf = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf.fit(X, y)
razdel_preds = clf.predict(X_test)
razdel_f1 = f1_score(y_test, razdel_preds, average='weighted')

print(classification_report(y_test, razdel_preds))

              precision    recall  f1-score   support

         0.0       0.80      0.76      0.78       949
         1.0       0.58      0.63      0.60       493

    accuracy                           0.72      1442
   macro avg       0.69      0.69      0.69      1442
weighted avg       0.72      0.72      0.72      1442



In [50]:
if default_f1 > razdel_f1:
    winner = "sklearn"
elif razdel_f1 > default_f1:
    winner = "razdel"
else:
    winner = "Draw"
    
print(f"Победил: {winner}")

Победил: razdel


## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

**CountVectorizer**

In [62]:
vectorizer = CountVectorizer(lowercase=True, min_df=3, max_df=0.85, ngram_range=(1, 2), strip_accents='unicode', max_features=10000)
X1 = vectorizer.fit_transform(train.comment)
X1_test = vectorizer.transform(test.comment)

clf1 = MultinomialNB(alpha=0.5, fit_prior=True, force_alpha=True)
clf1.fit(X1, y)
preds1 = clf1.predict(X1_test)
CV_f1 = f1_score(y_test, preds1, average='weighted')

print(classification_report(y_test, preds1))

              precision    recall  f1-score   support

         0.0       0.88      0.93      0.91       949
         1.0       0.85      0.77      0.81       493

    accuracy                           0.87      1442
   macro avg       0.87      0.85      0.86      1442
weighted avg       0.87      0.87      0.87      1442



**TfidfVectorizer**

In [64]:
vectorizer = TfidfVectorizer(lowercase=True, min_df=2, max_df=0.9, ngram_range=(2, 3), strip_accents='unicode')
X2 = vectorizer.fit_transform(train.comment)
X2_test = vectorizer.transform(test.comment)

clf2 = LogisticRegression(C=1.0, class_weight='balanced', max_iter=1000)
clf2.fit(X, y)
preds2 = clf2.predict(X_test)
TF_f1 = f1_score(y_test, preds2, average='weighted')

print(classification_report(y_test, preds2))

              precision    recall  f1-score   support

         0.0       0.81      0.73      0.77       949
         1.0       0.56      0.67      0.61       493

    accuracy                           0.71      1442
   macro avg       0.69      0.70      0.69      1442
weighted avg       0.72      0.71      0.71      1442



**10 самых токсичных текстов**

In [71]:
probs1 = clf1.predict_proba(X1_test)[:, 1]
probs2 = clf2.predict_proba(X2_test)[:, 1]

results_df = pd.DataFrame({
     'text': list(test.comment),
     'y_true': list(y_test),
     'CV_pred': preds1,
     'CV_prob': probs1,
     'TF_pred': preds2, 
     'TF_prob': probs2
})

CV_toxic_top10 = results_df.nlargest(10, 'CV_prob')[['text', 'CV_prob', 'y_true']]
for i, (idx, row) in enumerate(CV_toxic_top10.iterrows(), 1):
    print(f"{i}. [True: {row['y_true']}, Prob: {row['CV_prob']:.4f}] {row['text']}...")

TF_toxic_top10 = results_df.nlargest(10, 'TF_prob')[['text', 'TF_prob', 'y_true']]
for i, (idx, row) in enumerate(TF_toxic_top10.iterrows(), 1):
    print(f"{i}. [True: {row['y_true']}, Prob: {row['TF_prob']:.4f}] {row['text']}...")

1. [True: 1.0, Prob: 1.0000] Поясняю за Била. Те, кто его смотрит отморозки, или быдло. Они просто не могут представить, что могут оказаться в такой ситуации. А для них омежки это скот, но когда затрагивают их ЧСВ совершенно безобидным пранком, когда облили водой раздвиженцев, то быдло и отморозки встают в позу и говорят о НАРУШЕНИЕ ГРАНИЦ. Тем, кому припекает омежки, которые из-за низкой самооценки неосознанно представляют себя на месте героев пранка. А потом в своих влажных фантазия избивают этого пранкера или пытают его. А нормальные люди просто это не смотрят. А если и посмотрят то просто скажут, что он дурак. Бил же трус и быдло -- доебывается обычно до слабых, что свойственно гопоте. Очень удивилась, что у него так много лайков на видосах. Ну видимо это те, кто любят Хованского, Мопса и прочих быдлоютуберов. Не удивлюсь, что Била смотрят в основном не из приличных городов, а из всяких мухасрансков. Вывод, русские терпилы будут терпеть до конца. Как лев против или стопхам(который 

Отобрались разные тексты, но и там, и там они токсичные. Второй классификатор ошибся в предложении 2. Первый справился прекрасно!

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [124]:
nltk.download('stopwords')

russian_stopwords = stopwords.words('russian')

vectorizer = CountVectorizer(
    lowercase=True,
    min_df=2,
    max_df=0.9,
    ngram_range=(1, 2),
    strip_accents='unicode',
    max_features=5000,
    stop_words=russian_stopwords  # Используем русские стоп-слова
)

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

feature_names = vectorizer.get_feature_names_out()

def get_top_features(clf, feature_names, n=10):
    
    if hasattr(clf, 'coef_'):  # Logistic Regression
        importance = clf.coef_[0]
    elif hasattr(clf, 'feature_importances_'):  # Tree-based
        importance = clf.feature_importances_
    elif hasattr(clf, 'feature_log_prob_'):  # NB
        toxic_probs = np.exp(clf.feature_log_prob_[1])  # 1 - токсичный
        non_toxic_probs = np.exp(clf.feature_log_prob_[0])  # 0 - нетоксичный
        importance = toxic_probs / non_toxic_probs  # отношение вероятностей
    else:
        raise ValueError

    indices = np.argsort(importance)[::-1]

    top_features = []
    for i in range(min(n, len(indices))):
        idx = indices[i]
        feature_name = feature_names[idx]
        feature_importance = importance[idx]
        top_features.append((feature_name, feature_importance))

    return top_features

models = {}
top_features_all = {}


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\MSI\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['другои', 'еи', 'какои', 'мои', 'неи', 'сеичас', 'такои', 'этои'] not in stop_words.
  warnings.warn(


In [125]:
lr_model = LogisticRegression(
    C=0.8,
    class_weight='balanced',
    max_iter=1000
)

lr_model.fit(X, y)
lr_preds = lr_model.predict(X_test)
lr_f1 = f1_score(y_test, lr_preds, average='weighted')

models['LogisticRegression'] = lr_model
lr_f1

0.8183289884549476

In [145]:
dt_model = DecisionTreeClassifier(
    max_depth=800,
    min_samples_split=20,
    min_samples_leaf=4,
    class_weight='balanced',
)

dt_model.fit(X, y)
dt_preds = dt_model.predict(X_test)
dt_f1 = f1_score(y_test, dt_preds, average='weighted')

models['DecisionTree'] = dt_model
dt_f1

0.7236259663613449

In [149]:
nb_model = MultinomialNB(
    alpha=0.7,
    fit_prior=True
)

nb_model.fit(X, y)
nb_preds = nb_model.predict(X_test)
nb_f1 = f1_score(y_test, nb_preds, average='weighted')

models['MultinomialNB'] = nb_model
nb_f1

0.8482547090626886

In [155]:
rf_model = RandomForestClassifier(
    n_estimators=1000,
    max_depth=500,
    min_samples_split=30,
    min_samples_leaf=3,
    class_weight='balanced',
    n_jobs=-1
)

rf_model.fit(X, y)
rf_preds = rf_model.predict(X_test)
rf_f1 = f1_score(y_test, rf_preds, average='weighted')

models['RandomForest'] = rf_model
rf_f1

0.7895957041763683

In [156]:
for model_name, model in models.items():
    top_features = get_top_features(model, feature_names, n=5)
    for i, (feature, importance) in enumerate(top_features, 1):
        print(f"  {i}. {feature}: {importance:.4f}")

  1. хохлов: 2.9219
  2. дебил: 2.8375
  3. хохлы: 2.7111
  4. тупые: 2.2623
  5. дебилов: 2.1911
  1. хохлы: 0.0240
  2. тебе: 0.0233
  3. очень: 0.0228
  4. хохлов: 0.0202
  5. нахуи: 0.0185
  1. очень: 0.0169
  2. хохлы: 0.0145
  3. хохлов: 0.0132
  4. тебе: 0.0131
  5. например: 0.0112
  1. сука: 229.1228
  2. дебил: 184.6722
  3. хохлов: 154.2461
  4. терпи: 148.3035
  5. хохол: 144.2625
